In [51]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import confusion_matrix, matthews_corrcoef, recall_score, precision_score, f1_score
from scipy.stats import pearsonr
from sklearn.feature_selection import SelectKBest, f_classif

feature_types = ['AAC', 'APAAC', 'CTD', 'DPC', 'PAAC']

data_dir = '/home/darshana/Projects/druggable_proteins/processed_dataset'
feature_engineered_data_dir = '/home/darshana/Projects/druggable_proteins/feature_engineered_dataset'

# Concatenate features from different types
train_dataframes = []
test_dataframes = []

for feature_type in feature_types:
    train_data = pd.read_csv(f'{data_dir}/TR_{feature_type}.csv')
    test_data = pd.read_csv(f'{data_dir}/TS_{feature_type}.csv')
    train_dataframes.append(train_data.set_index('id'))
    test_dataframes.append(test_data.set_index('id'))

# concat dataframes based on equal column ids
df_train = pd.concat(train_dataframes, axis=1, join='inner')
df_test = pd.concat(test_dataframes, axis=1, join='inner')
X_train = df_train.drop(columns=['label'], axis=1)
y_train = df_train['label'].loc[:, ~df_train['label'].columns.duplicated()]

# Initialize SelectKBest class
selector = SelectKBest(f_classif, k=100)

# Fit and transform the data
X_train_new = selector.fit_transform(X_train, y_train)

# Get columns to keep
cols = selector.get_support(indices=True)

# Create new dataframe with only desired columns, or overwrite existing
selected_features_df = X_train.iloc[:,cols]
selected_features_df['label'] = y_train
selected_features_df.reset_index(inplace=True)
df_test.reset_index(inplace=True)
df_test = df_test[selected_features_df.columns].loc[:, ~df_test[selected_features_df.columns].columns.duplicated()]

# Save the dataframe
selected_features_df.to_csv(f'{feature_engineered_data_dir}/TR_selected_features_all_best100.csv', index=False)
df_test.to_csv(f'{feature_engineered_data_dir}/TS_selected_features_all_best100.csv', index=False)

/home/darshana/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/tmp/ipykernel_63817/3864852222.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_features_df['label'] = y_train
